In [1]:
!nodetool status

Datacenter: datacenter1
Status=Up/Down
|/ State=Normal/Leaving/Joining/Moving
--  Address     Load       Tokens  Owns (effective)  Host ID                               Rack 
UN  172.21.0.4  70.26 KiB  16      68.1%             d01fe928-848d-44bc-89b6-d4278c52e0b2  rack1
UN  172.21.0.2  70.28 KiB  16      60.8%             22c25040-5cc2-4a41-9aa0-75aa20c93ef9  rack1
UN  172.21.0.3  70.26 KiB  16      71.0%             fe0faafc-6d38-4b16-a0e8-20d188a524f7  rack1

